In [ ]:
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis, linkage_optimization
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

In [ ]:
import parallelism
parallelism.set_max_num_tbb_threads(24)
parallelism.set_hessian_assembly_num_threads(8)

In [ ]:
# Reloadable libraries
import importlib

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

import vis.fields
import matplotlib.cm as cm

In [ ]:
# Fert 1
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [2 * 1/800, 1/800]
MODEL_NAME = "fert_1"
MODEL_PATH = osp.join(weaving_dir + 'normalized_objs/models/{}.obj'.format(MODEL_NAME))
SUBDIVISION_RESOLUTION = 10
SMOOTHING_WEIGHT = 10
REGULARIZATION_WEIGHT = 0
INPUT_SURFACE_PATH = osp.join(weaving_dir + 'normalized_objs/surface_models/{}.obj'.format(MODEL_NAME))
RIBBON_NAME = "{}_strip".format(MODEL_NAME)

In [ ]:
def initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH):
    l = elastic_rods.SurfaceAttractedLinkage(surface_path, useCenterline, model_path, subdivision_res, False, InterleavingType.triaxialWeave)
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    l.setDoFs(l.getDoFs())
    l.set_holdClosestPointsFixed(True);
    l.set_attraction_tgt_joint_weight(0.01);
    l.attraction_weight = 100;
    return l

In [ ]:
elastic_rods.benchmark_reset()
cl2 = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)
elastic_rods.benchmark_report()

In [ ]:
def initialize_normal_linkage(cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH):
    l = elastic_rods.RodLinkage(model_path, subdivision_res, False)
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    return l

In [ ]:
def design_parameter_solve(l,regularization_weight = 0.1, smoothing_weight = 1):
    design_opts = elastic_rods.NewtonOptimizerOptions()
    design_opts.niter = 10000
    design_opts.verbose = 10
    l.set_design_parameter_config(use_restLen = True, use_restKappa = True)
    elastic_rods.designParameter_solve(l, design_opts, regularization_weight = regularization_weight, smoothing_weight = smoothing_weight)
    l.set_design_parameter_config(use_restLen = True, use_restKappa = True)

In [ ]:
def get_linkage_eqm(l, opt, cam_param = default_camera_parameters, target_surf = None):
    elastic_rods.compute_equilibrium(l, options = opt)
    if (target_surf is None):
        view = linkage_vis.LinkageViewer(l, width=1024, height=640)
    else:
        view = linkage_vis.LinkageViewerWithSurface(l, target_surf, width=1024, height=640)
    view.setCameraParams(cam_param)
    return l, view

In [ ]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 10;
OPTS.beta = 1e-8
OPTS.niter = 100
OPTS.verboseNonPosDef = False

### Create Surface Attracted Linkage and solve for design parameters with fixed joint positions

In [ ]:
with so(): curved_linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();

In [ ]:
def applyViewStyle(v):
    v.viewOptions[v.ViewType.LINKAGE].color = '#28D'
    v.viewOptions[v.ViewType.LINKAGE].transparent = False
    v.viewOptions[v.ViewType.SURFACE].transparent = True
    v.viewOptions[v.ViewType.SURFACE].color = 'gray'
    v.applyViewOptions()

In [ ]:
curved_linkage_view = linkage_vis.LinkageViewerWithSurface(curved_linkage, INPUT_SURFACE_PATH, width=1024, height=640)
applyViewStyle(curved_linkage_view)
curved_linkage_view.show()

In [ ]:
def curved_callback(prob, i):
    #curved_linkage_view.update()
    pass

In [ ]:
curved_linkage_view.update()

In [ ]:
fixed_joint_orientation = []
for i in range(curved_linkage.numJoints()):
    fixed_joint_orientation.extend(list(range(curved_linkage.dofOffsetForJoint(i) + 3, curved_linkage.dofOffsetForJoint(i) + 6)))

In [ ]:
rlw = 1
rcw = 1e-5
curved_dpo = elastic_rods.get_designParameter_optimizer(curved_linkage, rlw, rcw, callback=curved_callback, fixedVars = fixed_joint_orientation)
curved_dpo.options.niter = 200
curved_dpp = curved_dpo.get_problem()

curved_dpp.set_regularization_weight(1)
curved_dpp.set_smoothing_weight(1e-4)

In [ ]:
# Run first with a less extreme ribbon aspect ratio (more stable)
curved_linkage.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, [2 / 800, 1 / 800], stiffAxis=elastic_rods.StiffAxis.D1))

elastic_rods.benchmark_reset()
curved_cr = curved_dpo.optimize()
elastic_rods.benchmark_report()

In [ ]:
# Now, run with a wide cross-section
# Also free the constraints on the joint orientation
curved_dpo.setFixedVars(curved_linkage.restLenFixedVars())
curved_linkage.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, [8 / 800, 1 / 800], stiffAxis=elastic_rods.StiffAxis.D1))
elastic_rods.benchmark_reset()
curved_cr = curved_dpo.optimize()
elastic_rods.benchmark_report()

In [ ]:
curved_linkage.attraction_weight = 1e-1

In [ ]:
OPTS.verbose = 1

In [ ]:
elastic_rods.compute_equilibrium(curved_linkage, options = OPTS)
curved_linkage_view.update()

In [ ]:
import linkage_optimization

In [ ]:
OPTS.niter = 200
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
useCenterline = True
optimizer = linkage_optimization.WeavingOptimization(curved_linkage, INPUT_SURFACE_PATH, useCenterline, equilibrium_options=OPTS, pinJoint = 0, useFixedJoint = False)
optimizer.set_target_joint_position(curved_save_tgt_joint_pos)

In [ ]:
view = linkage_vis.LinkageViewerWithSurface(curved_linkage, INPUT_SURFACE_PATH, width=1024, height=640)
applyViewStyle(view)

In [ ]:
view.show()

In [ ]:
optimizer.rl_regularization_weight = 0
optimizer.smoothing_weight = 1e-2
optimizer.beta = 5000000.0
optimizer.gamma = 1
algorithm = linkage_optimization.WeavingOptAlgorithm.NEWTON_CG
def update_viewer():
    view.update()

In [ ]:
for t in optimizer.objective.terms:
    print(f'{t.name}: {t.term.value()}')

In [ ]:
elastic_rods.benchmark_reset()
optimizer.WeavingOptimize(algorithm, 50, 1.0, 1e-2, update_viewer)
elastic_rods.benchmark_report()

In [ ]:
optimizer.setLinkageAttractionWeight(1e-6)

In [ ]:
OPTS.gradTol = 1e-7
with so(): elastic_rods.compute_equilibrium(curved_linkage, options = OPTS)
view.update()

In [ ]:
# Verify that the elastic forces are essentially in balance on their own
np.linalg.norm(curved_linkage.gradient(energyType=elastic_rods.SurfaceAttractionEnergyType.Elastic))

In [ ]:
np.linalg.norm(curved_linkage.gradient(energyType=elastic_rods.SurfaceAttractionEnergyType.Attraction))

In [ ]:
curved_linkage.saveVisualizationGeometry('optimized.msh')